In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from speckit import compute_spectrum
import speckit.core as _core
import speckit.analysis as _analysis

# --- helpers to toggle numba path globally in speckit ------------------------
def _set_numba_enabled(flag: bool):
    _core._NUMBA_ENABLED = bool(flag)
    _analysis._NUMBA_ENABLED = bool(flag)

# --- config ------------------------------------------------------------------
N_values = np.array([1e3, 1e4, 1e5, 1e6], dtype=int)

orders = [-1, 0, 1, 2]
times_numba = {order: [] for order in orders}
times_numpy = {order: [] for order in orders}

# Inputs
inputs = []
np.random.seed(123)
for N in N_values:
    inputs.append(np.random.rand(N))

# --- warm-up: compile numba once so timing isn't dominated by JIT ------------
_set_numba_enabled(True)
_ = compute_spectrum(inputs[0], fs=2, order=0)

# --- benchmark: Numba path ---------------------------------------------------
for order in orders:
    print(f"[Numba] order={order}")
    _set_numba_enabled(True)
    for x in inputs:
        t0 = time.time()
        _ = compute_spectrum(x, fs=2, order=order)
        times_numba[order].append(time.time() - t0)

# --- benchmark: NumPy fallback path ------------------------------------------
for order in orders:
    print(f"[NumPy] order={order}")
    _set_numba_enabled(False)
    for x in inputs:
        t0 = time.time()
        _ = compute_spectrum(x, fs=2, order=order)
        times_numpy[order].append(time.time() - t0)

# --- quick numerical consistency check (choose an index into N_values) -------
check_idx = 1  # 0..len(N_values)-1; here N=1e4
x_check = inputs[check_idx]

print("\nNumerical consistency check (Numba vs NumPy):")
for order in orders:
    # Numba result
    _set_numba_enabled(True)
    res_nb = compute_spectrum(x_check, fs=2, order=order)
    # NumPy result
    _set_numba_enabled(False)
    res_np = compute_spectrum(x_check, fs=2, order=order)

    # Compare Gxx; also ASD as a sanity check
    gxx_nb, gxx_np = res_nb.Gxx, res_np.Gxx
    asd_nb, asd_np = res_nb.asd, res_np.asd

    def _rel_err(a, b, eps=1e-300):
        num = np.max(np.abs(a - b))
        den = np.max(np.abs(b)) + eps
        return num / den

    rel_gxx = _rel_err(gxx_nb, gxx_np)
    rel_asd = _rel_err(asd_nb, asd_np) if asd_nb is not None else np.nan

    print(f"  order={order}: max rel err Gxx={rel_gxx:.3e}, ASD={rel_asd:.3e}")

In [ ]:
# --- plot timings ------------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 3.5), dpi=150)
markers = { -1:"o", 0:"s", 1:"^", 2:"D" }

for order in orders:
    ax.loglog(N_values, times_numba[order], marker=markers[order], ls="-",  label=f"Numba  (order={order})")
    ax.loglog(N_values, times_numpy[order], marker=markers[order], ls="--", label=f"NumPy   (order={order})")

ax.set_xlabel("Length of time series N")
ax.set_ylabel("Computation time (s)")
ax.legend(
    loc="upper left",
    bbox_to_anchor=(1, 1),
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    ncol=2,
)
ax.grid(True, which="both", color="lightgray", linestyle="-", linewidth=0.5)
fig.tight_layout()
plt.show()